In [ ]:
import dash
import dash_leaflet as dl
from dash import dcc, html, dash_table as dt
from dash.dependencies import Input, Output
import plotly.express as px
import base64
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

# Import AnimalShelter class
from animalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
animals = AnimalShelter()  # Local MongoDB Compass, no username/password required

# Removed full database read so it can be optimized for performance
# df = pd.DataFrame.from_records(animals.read({})) has been removed

# Created empty DataFrame for initial layout
df = pd.DataFrame(columns=[
    "animal_id", "name", "animal_type", "breed", "age_upon_outcome",
    "outcome_type", "sex_upon_outcome", "age_upon_outcome_in_weeks",
    "location_lat", "location_long"
])

#########################
# Dashboard Layout / View
#########################
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# Encode image to display logo
image_filename = 'Grazioso_Salvare_Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display': 'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.B(html.H2('Harrielle Monestime'))),
    html.Hr(),
    html.Center(html.Img(
        id='customer-image',
        src='data:image/png;base64,{}'.format(encoded_image),
        alt='Grazioso Salvare Logo')),
    html.Hr(),

    # Filter Options
    html.Div([
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'mount'},
                {'label': 'Disaster Rescue and Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset'
        )
    ]),

    html.Hr(),

    # Data Table
    dt.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=False,
        sort_action="native",
        sort_mode="multi",
        column_selectable="single",
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current=0,
        page_size=10,
    ),

    html.Br(),
    html.Hr(),

    # Graph + Map
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

#######################
# Callbacks
#######################

@app.callback(
    [Output('datatable-id', 'data'), Output('datatable-id', 'columns')],
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    if filter_type == 'water':
        filtered = animals.read({
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte": 156.0}
        })
    elif filter_type == 'mount':
        filtered = animals.read({
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte": 156.0}
        })
    elif filter_type == 'disaster':
        filtered = animals.read({
            "animal_type": "Dog",
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20.0, "$lte": 300.0}
        })
    else:
        filtered = animals.read({})  # Reset loads all records

    df_filtered = pd.DataFrame.from_records(filtered)
    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df_filtered.columns]
    data = df_filtered.to_dict('records')

    return data, columns

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if viewData is None or len(viewData) == 0:
        return [html.P("No data available for visualization.")]

    dff = pd.DataFrame.from_dict(viewData)
    
    if 'breed' not in dff.columns or dff['breed'].dropna().empty:
        return [html.P("Breed column missing or empty.")]
    
    return [
        dcc.Graph(figure=px.histogram(dff, x='breed', title='Distribution of Breeds'))
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_map(viewData, selected_rows):
    dff = pd.DataFrame.from_dict(viewData)

    if selected_rows is None or len(selected_rows) == 0:
        return []

    selected_row = selected_rows[-1]
    lat = dff.iloc[selected_row, 13]
    lon = dff.iloc[selected_row, 14]

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[lat, lon], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(dff.iloc[selected_row, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[selected_row, 9])
                ])
            ])
        ])
    ]

if __name__ == '__main__':
    app.run(debug=True)
